In [25]:
import sys
import torch
from torch import nn

sys.path.append("../") # if running from this folder
from src.models import NAVAR
from src.data import load_dataset

In [14]:
dataset = load_dataset('causeme', 'nonlinear-VAR_N-5_T-300')[0]
dataset

{'name': 'nonlinear-VAR_N-5_T-300',
 'data': tensor([[[-2.7631, -2.1176, -0.6967,  ...,  0.9088,  0.4479,  1.3872],
          [ 1.0702,  0.9536,  1.1921,  ...,  0.5790, -0.9674, -1.3237],
          [ 1.8066,  2.3471,  1.7101,  ...,  0.0251, -0.5003,  0.1296],
          [ 1.2085,  0.5059, -0.0125,  ...,  1.1270, -1.6082, -2.0201],
          [-0.4758, -0.2494, -0.4014,  ..., -0.3803, -0.0652, -0.2039]]])}

In [23]:
tensor_data = dataset['data']
print(tensor_data.shape)

torch.Size([1, 5, 300])


In [30]:
ground_truth = (torch.randn(5, 5) > 0).int() # fake ground truth data
ground_truth

tensor([[1, 1, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0]], dtype=torch.int32)

In [20]:
navar_params = {
    'n_variables': 5,
    'hidden_dim': 8,
    'lambda1': 0.2 # regularization
}
tcn_params = {
    'kernel_size': 3,
    'n_blocks': 4,
    'n_layers': 2,
    'dropout': 0.2,
}

In [24]:
model = NAVAR(**navar_params, **tcn_params)

if torch.cuda.is_available():
    tensor_data = tensor_data.cuda()
    model = model.cuda()
    
model

NAVAR(
  (contributions): TCN(
    (tcn): DefaultTCN(
      (network): Sequential(
        (0): TemporalBlock(
          (dropout): Dropout(p=0.2, inplace=False)
          (relu): ReLU()
          (convolutions): ModuleList(
            (0): ParametrizedConv1d(
              5, 40, kernel_size=(3,), stride=(1,), groups=5
              (parametrizations): ModuleDict(
                (weight): ParametrizationList(
                  (0): _WeightNorm()
                )
              )
            )
            (1): ParametrizedConv1d(
              40, 40, kernel_size=(3,), stride=(1,), groups=5
              (parametrizations): ModuleDict(
                (weight): ParametrizationList(
                  (0): _WeightNorm()
                )
              )
            )
          )
        )
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): TemporalBlock(
          (dropout): Dropout(p=0.2, inplace=False)
          (relu): ReLU()
          (convolutions): ModuleL

## Forward function
The forward function can return metrics (singular values such as loss and AUC) and artifacts (generated tensors in the process, such as causal_matrix etc)

In [27]:
# return metrics data (loss AUC etc.)
model(tensor_data)

({'loss': tensor(1.8632, device='cuda:0', grad_fn=<AddBackward0>)}, {})

In [28]:
# create_artifacts will return artifacts alongside metrics (contributions, causal matrix etc.)
model(tensor_data, create_artifacts=True)

({'loss': tensor(1.8513, device='cuda:0', grad_fn=<AddBackward0>)},
 {'prediction': tensor([[[-0.8910, -0.6843,  0.5165,  ..., -1.0437, -1.0180, -0.1225],
           [-1.2888,  0.3118, -0.0852,  ..., -0.5080, -0.3155, -0.2926],
           [-0.1682,  1.4510,  0.8662,  ..., -0.0629,  1.2251,  0.6010],
           [ 1.5773,  0.8131,  0.8425,  ...,  0.6957,  0.4989,  1.6930],
           [ 1.0173,  0.3299, -0.2290,  ...,  1.3274,  1.1022,  0.5172]]],
         device='cuda:0'),
  'contributions': tensor([[[[ 0.1490, -0.4650,  0.4059,  ..., -0.7896, -0.6438, -0.1478],
            [-0.7267, -0.5505, -0.4943,  ..., -0.2782, -0.7143, -0.2781],
            [-0.2044,  0.0351,  0.0949,  ..., -0.0132,  0.0342,  0.1902],
            [ 0.0906,  0.2168,  0.4575,  ...,  0.0356,  0.1369,  0.2028],
            [-0.1762,  0.1024,  0.0758,  ...,  0.0249,  0.1923, -0.0664]],
  
           [[ 0.0063,  0.7212,  0.6394,  ...,  0.1125,  0.2133,  0.1600],
            [-0.4971, -0.3310, -0.3799,  ..., -0.0343, -0.1

In [29]:
# temporal_matrix will apply a sliding window std to construct a temporal matrix
model(tensor_data, create_artifacts=True, temporal_matrix=True)

({'loss': tensor(1.8668, device='cuda:0', grad_fn=<AddBackward0>)},
 {'prediction': tensor([[[-0.7762, -0.4773,  0.0165,  ..., -0.2425, -0.6669, -0.2366],
           [-1.5769, -0.8642, -0.3292,  ..., -0.1031, -0.6567, -0.7526],
           [ 0.0087,  0.5575,  0.1983,  ...,  0.9405,  0.7223,  0.1413],
           [ 0.5796,  0.9557,  0.9467,  ...,  0.2904,  0.5084,  1.8234],
           [ 1.8582,  1.2192,  0.7135,  ...,  0.6314,  0.8372,  0.4071]]],
         device='cuda:0'),
  'contributions': tensor([[[[-0.7677, -0.5064, -0.2708,  ..., -0.3606, -0.3320, -0.0115],
            [-0.4177, -0.5156, -0.2574,  ..., -0.2712, -0.5752, -0.5086],
            [ 0.0115,  0.2929,  0.2498,  ...,  0.3019,  0.1131,  0.1537],
            [ 0.1688,  0.2943,  0.3736,  ...,  0.0595,  0.0385,  0.2288],
            [ 0.2521, -0.0193, -0.0554,  ...,  0.0512,  0.1119, -0.0758]],
  
           [[ 0.2535,  0.1356,  0.4268,  ...,  1.0680,  0.3821,  0.2943],
            [-0.5509, -0.3257, -0.2181,  ..., -0.3091, -0.3

In [31]:
# ground_truth will compute AUC etc. and adds it to metrics
model(tensor_data, create_artifacts=True, ground_truth=ground_truth)

({'loss': tensor(1.8967, device='cuda:0', grad_fn=<AddBackward0>),
  'AUROC': tensor(0.5390, device='cuda:0')},
 {'prediction': tensor([[[-0.2545, -0.4783, -0.9281,  ..., -0.6489, -0.5933, -0.1058],
           [-0.7596, -0.6367, -1.3251,  ..., -0.5310, -0.1681, -0.0706],
           [ 0.6276, -0.1577, -0.2623,  ...,  0.0513,  0.5668, -0.1659],
           [ 1.2374,  1.4031,  0.5129,  ...,  0.9824,  1.6823,  1.4160],
           [ 0.4674,  1.3683,  2.1789,  ...,  0.7053, -0.1676,  0.6918]]],
         device='cuda:0'),
  'contributions': tensor([[[[ 0.3924, -0.1518, -0.9459,  ..., -0.2920, -0.0943, -0.0138],
            [-0.5296, -0.3997, -0.2708,  ..., -0.4178, -0.5906, -0.1948],
            [-0.2175, -0.1230, -0.0240,  ...,  0.0435,  0.1460,  0.0760],
            [ 0.2367,  0.1643,  0.3980,  ..., -0.0572,  0.1654,  0.1614],
            [-0.1133,  0.0553, -0.0622,  ...,  0.0979, -0.1966, -0.1114]],
  
           [[ 0.4026,  0.0458, -0.4155,  ...,  0.1537,  0.2319,  0.5313],
            [-0